In [1]:
# add path to sispeuede to sys.path in python
import sys
import pathlib
import warnings
warnings.filterwarnings("ignore")


path_git = pathlib.Path("/Users/usuario/git")
for subdir in [
    "sisepuede", 
    "sisepuede_data_pipeline",
    "sisepuede_juypyter"
]:
    _PATH_CUR = path_git.joinpath(subdir)
    if str(_PATH_CUR) not in sys.path:
        sys.path.append(str(_PATH_CUR))

path_pipeline = path_git.joinpath("sisepuede_data_pipeline")


import importlib
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import sisepuede.core.attribute_table as att
import sisepuede.core.support_classes as sc
import sisepuede.legacy.data_api as api
import sisepuede.manager.sisepuede_examples as sxl
import sisepuede.manager.sisepuede_file_structure as sfs
import sisepuede.manager.sisepuede_models as sm
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf
import sisepuede.visualization.plots as svp


import time
from typing import *

# from sisepuede_data_pipeline
import lib.process_utilities as pu
import lib.sisepuede_data_constructs as dc
import lib._util as lutil

# from sisepuede_jupyter
import temp_update_fields_from_wv_to_main as temp 


# Setup SISEPUEDE elements

In [2]:
def get_file_structure(
    y0: int = 2015,
    y1: int = 2070,
) -> Tuple[sfs.SISEPUEDEFileStructure, att.AttributeTable]:
    """Get the SISEPUEDE File Structure and update the attribute table
        with new years.
    """
    # setup some SISEPUEDE variables and update time period
    file_struct = sfs.SISEPUEDEFileStructure(
        initialize_directories = False,
    )

    # get some keys
    key_time_period = file_struct.model_attributes.dim_time_period
    key_year = file_struct.model_attributes.field_dim_year


    ##  BUILD THE ATTRIBUTE AND UPDATE

    # setup the new attribute table
    years = np.arange(y0, y1 + 1, ).astype(int)
    attribute_time_period = att.AttributeTable(
        pd.DataFrame(
            {
                key_time_period: range(len(years)),
                key_year: years,
            }
        ),
        key_time_period,
        
    )

    # finally, update the ModelAttributes inside the file structure
    (
        file_struct
        .model_attributes
        .update_dimensional_attribute_table(
            attribute_time_period,
        )
    )

    # return the tuple
    out = (file_struct, attribute_time_period, )

    return out

    
# set up some paths
_PATH_CUR = pathlib.Path(os.getcwd())
_PATH_DATA = _PATH_CUR.joinpath("data")
_PATH_OUTPUT = _PATH_DATA.joinpath("output")

# export prefix
_PREFIX_FILENAME_DATASETBUILD_BY_REGION = "sisepuede_raw_inputs_latest_"


# model attributes and associated support classes
_EXAMPLES = sxl.SISEPUEDEExamples()
_FILE_STRUCTURE, _ATTRIBUTE_TABLE_TIME_PERIOD = get_file_structure()
matt = _FILE_STRUCTURE.model_attributes
regions = sc.Regions(matt, )
time_periods = sc.TimePeriods(matt, )

# setup models
models = sm.SISEPUEDEModels(
    matt,
    allow_electricity_run = True,
    fp_julia = _FILE_STRUCTURE.dir_jl,
    fp_nemomod_reference_files = _FILE_STRUCTURE.dir_ref_nemo,
    initialize_julia = True, 
)

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   4270.6 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.


In [57]:
##  SOME GLOBALS

df_example_input = _EXAMPLES("input_data_frame")
tab = regions.attributes.table


##  NOTE! Need to ignore these for the moment
# The GDP for these are incomplete:
#   - antigua_and_barbuda
#   - cayman_islands
# There are other issues with these countries:
#   - british_virgin_islands
#   - cuba
#   - curacao
#   - dominica
#   - grenada'
#   - saint_kitts_and_nevis
#   - saint_martin
#   - sint_maarten
#   - turks_and_caicos_islands
#   - united_states_virgin_islands
#
_REGIONS_TO_CHECK = [
    "antigua_and_barbuda", 
    "british_virgin_islands",
    "cayman_islands",
    "cuba",
    "curacao",
    "dominica",
    "grenada",
    "saint_kitts_and_nevis",
    "saint_martin",
    "sint_maarten",
    "turks_and_caicos_islands",
    "united_states_virgin_islands"
]

_REGIONS_BUILD = list(
    tab[
        tab["un_sub_region"].isin(["Latin America and the Caribbean"])
        & ~tab[regions.key].isin(_REGIONS_TO_CHECK)    
    ][regions.key]
    .unique()
)
_REGIONS_BUILD = sorted(
    _REGIONS_BUILD +
    [
        "bulgaria",
        "egypt",
        "libya",
        "morocco",
        "united_republic_of_tanzania"
    ]
)


_REGIONS_ISO = [
    regions.return_region_or_iso(x, return_type = "iso", )
    for x in _REGIONS_BUILD
]


# Setup old repository and read data

In [5]:
repo_old = api.SISEPUEDEBatchDataRepository(
    "/Users/usuario/git/sisepuede_data", 
    matt,
)

print("Getting old repository data...")
df_old = repo_old.read(None)
print("Old repository data complete.")

Getting old repository data...
Old repository data complete.


# Setup new repository

In [58]:
importlib.reload(dc)
construct = dc.SISEPUEDEDataConstructs(
    path_output_database = "/Users/usuario/git/sisepuede_data_pipeline/sisepuede_inputs.sqlite",
)

path_repo = pathlib.Path("/Users/usuario/SISEPUEDE_DATA_REPOSITORY")
repo = pu.Repository(
    {
        "local": {
            "path": str(path_repo)
        }
    }
)

# get from pipeline
df_from_pipeline = construct.build_inputs_from_database(
    regions_keep = _REGIONS_BUILD,
    join = "outer"
)


In [69]:

def function_combine(
    df_repo_new: pd.DataFrame,
    df_repo_old: pd.DataFrame,
    df_example: pd.DataFrame,
    region_iso: str,
    years: Union[List[int], None] = None,
) -> pd.DataFrame:
    """Combine DataFrames by region, combining in a hierarchy
    """
    
    ##  FORMAT A BASE 

    # start by setting years
    if not sf.islistlike(years):
        years = time_periods.all_years

    
    df_base = (
        df_repo_old[
            df_repo_old[repo_old.field_repo_iso].isin([region_iso])
            & (df_repo_old[time_periods.field_year] >= min(years))
        ]
        .copy()
        .rename(
            columns = {
                repo_old.field_repo_iso: regions.field_iso,
                repo_old.field_repo_year: time_periods.field_year,
            }
        )
    )

    # QUICK FIX!
    if "pop_lvst_initial_chickens" in df_base.columns:
        df_base["pop_lvst_initial_chickens"] *= 1000
        print("NOTE! Adjusting poultry population")
        
    # cols_drop = [
    #     x for x in df_base.columns 
    #     if df_base[x].dropna().shape[0] != df_base.shape[0]
    # ]
    #
    # df_base = df_base.drop(columns = cols_drop, )
    df_base[time_periods.field_year] = df_base[time_periods.field_year].astype(int)    
    df_base = (
        pd.merge(
            pd.DataFrame({time_periods.field_year: years, }),
            df_base,
            how = "left",
        )
        .ffill()
        .bfill()
    )


    ##  ADD IN PIPELINE DATA

    df_out = sf.match_df_to_target_df(
        df_base, 
        df_from_pipeline,
        [
            construct.time_periods.field_year,
            regions.field_iso,
        ],
        overwrite_only = False,
    )
    
    df_out[time_periods.field_year] = df_out[time_periods.field_year].astype(int)
    df_out = (time_periods.years_to_tps(df_out, ))


    ##  PULL MISSING FIELDS FROM EXAMPLE DF
    
    # fields not in base
    fields_missing = [
        x for x in df_example.columns 
        if (x not in df_out.columns) 
        and (x in matt.all_variable_fields_input)
    ]
    
    # specify fields to pull from the example
    fields_from_ex = [
        x for x in fields_missing 
        if not (
            False#x.startswith("frac_lndu_")
            #x.startswith("factor_lndu")
            #or x.startswith("frac_lndu_")
        )
    ]
    
    fields_from_ex = [
        x for x in fields_from_ex
        if (x not in df_out.columns)
    ]

    # merge in from ex
    df_out = (
        pd.merge(
            df_out,
            df_example[fields_from_ex + [time_periods.field_time_period]],
            how = "left"
        )
        .ffill()
        .bfill()
    )


    ##  TEMPORARY SCRIPT FOR MOVING FROM working_version TO latest full version
    
    df_out = temp.update_fields(
        df_out,
        matt,
    )

    return df_out


df_mex = function_combine(
    df_from_pipeline,
    df_old,
    df_example_input,
    "MEX",
)



NOTE! Adjusting poultry population


# Run the next cell to build a composite file and/or export individual fiels (set `export = True` to do this)

In [71]:

df_out = []
export = True
# years = range(2015, 2071)

def build_path_for_region_file(
    iso: str,
) -> pathlib.Path:
    """Build the output path for a file
    """
    fn = f"{_PREFIX_FILENAME_DATASETBUILD_BY_REGION}{iso}.csv"
    path_out = _PATH_OUTPUT.joinpath(fn)
    
    return path_out



for iso in _REGIONS_ISO:

    df_cur = function_combine(
        df_from_pipeline,
        df_old,
        df_example_input,
        iso,
    )    
    df_cur[regions.field_iso] = iso

    # write to the output location?
    if export:
        path_write = build_path_for_region_file(iso, )
        df_cur.to_csv(
            path_write,
            encoding = "UTF-8",
            index = None,
        )
    
    df_out.append(df_cur,)
    print(f"Region {iso} complete")

df_out = sf._concat_df(df_out, )




NOTE! Adjusting poultry population
Region ARG complete
NOTE! Adjusting poultry population
Region ABW complete
NOTE! Adjusting poultry population
Region BHS complete
NOTE! Adjusting poultry population
Region BRB complete
NOTE! Adjusting poultry population
Region BLZ complete
NOTE! Adjusting poultry population
Region BOL complete
NOTE! Adjusting poultry population
Region BRA complete
NOTE! Adjusting poultry population
Region BGR complete
NOTE! Adjusting poultry population
Region CHL complete
NOTE! Adjusting poultry population
Region COL complete
NOTE! Adjusting poultry population
Region CRI complete
NOTE! Adjusting poultry population
Region DOM complete
NOTE! Adjusting poultry population
Region ECU complete
NOTE! Adjusting poultry population
Region EGY complete
NOTE! Adjusting poultry population
Region SLV complete
NOTE! Adjusting poultry population
Region GTM complete
NOTE! Adjusting poultry population
Region GUY complete
NOTE! Adjusting poultry population
Region HTI complete
NOTE! Adju

# option to test


In [303]:
region_run = "MEX"
df_run = models.project(
    df_out[
        df_out[regions.field_iso].isin([region_run])
    ].reset_index(drop = True),
    time_periods_base = np.arange(12),
    verbose = True,
)



2025-24-Sep 11:31:01.121 Started modeling scenario. NEMO version = 2.2.0, solver = HiGHS.
2025-24-Sep 11:31:01.128 Validated run-time arguments.
2025-24-Sep 11:31:01.131 Connected to scenario database. Path = /Users/usuario/git/sisepuede_region_nbs/generic_regions/tmpp8wwch9h.sqlite.
2025-24-Sep 11:31:01.275 Dropped pre-existing result tables from database.
2025-24-Sep 11:31:01.400 Created parameter views and indices.
2025-24-Sep 11:31:01.406 Created temporary tables.
2025-24-Sep 11:31:01.409 Started optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].
2025-24-Sep 11:31:01.411 Verified that transmission modeling is not enabled.
2025-24-Sep 11:31:01.429 Defined dimensions.
2025-24-Sep 11:31:04.463 Executed core database queries.
2025-24-Sep 11:31:04.468 Defined demand variables.
2025-24-Sep 11:31:04.470 Defined storage variables.
2025-24-Sep 11:31:04.470 Defined capacity variables.
2025-24-Sep 11:31:07.059 Defined activity variables.
2025

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,1.776620e+06,5.777571e+07,6.123765e+06,1.080999e+07,95945.934079,4.174639e+06,5.107607e+07,952640.985459,3.638277e+06,...,6.387951e+07,21702.395765,1.369572e+07,9.659114e+07,5.380426e+06,4.249135e+06,3.814442e+07,8.314306e+06,3.327182e+07,7.279251e+07
1,1,1.765776e+06,5.742306e+07,6.086387e+06,1.074401e+07,95360.306264,4.149158e+06,5.076432e+07,946826.324685,3.616070e+06,...,6.348220e+07,22642.119775,1.361212e+07,9.607292e+07,5.356823e+06,4.030275e+06,3.791160e+07,8.500211e+06,3.350091e+07,7.268603e+07
2,2,1.749462e+06,5.689253e+07,6.030155e+06,1.064474e+07,94479.273875,4.110824e+06,5.029531e+07,938078.610977,3.582662e+06,...,6.304530e+07,21006.136939,1.348636e+07,9.523104e+07,5.199789e+06,3.988301e+06,3.755597e+07,8.537499e+06,3.334806e+07,7.243672e+07
3,3,1.740842e+06,5.661219e+07,6.000441e+06,1.059229e+07,94013.721262,4.090568e+06,5.004748e+07,933456.169137,3.565008e+06,...,6.295853e+07,21120.255624,1.341990e+07,9.478523e+07,5.474726e+06,3.999300e+06,3.737625e+07,8.458248e+06,3.320639e+07,7.246789e+07
4,4,1.728610e+06,5.621440e+07,5.958279e+06,1.051786e+07,93353.130332,4.061825e+06,4.969582e+07,926897.204430,3.539958e+06,...,6.222427e+07,20714.593135,1.332561e+07,9.409320e+07,5.271402e+06,3.999685e+06,3.711097e+07,7.998781e+06,3.256082e+07,7.208738e+07
5,5,1.716749e+06,5.582870e+07,5.917397e+06,1.044570e+07,92712.607196,4.033956e+06,4.935484e+07,920537.491565,3.515669e+06,...,6.160202e+07,20163.672116,6.617089e+06,6.529472e+07,5.249084e+06,3.976892e+06,2.369336e+07,7.975904e+06,2.532098e+07,6.993747e+07
6,6,1.704969e+06,5.544561e+07,5.876793e+06,1.037402e+07,92076.427924,4.006275e+06,4.901617e+07,914220.908647,3.491545e+06,...,6.126594e+07,19974.563798,1.051487e+07,8.349278e+07,5.262042e+06,3.958070e+06,2.875984e+07,7.882006e+06,2.917020e+07,6.990728e+07
7,7,1.693269e+06,5.506514e+07,5.836466e+06,1.030283e+07,91444.591932,3.978784e+06,4.867982e+07,907947.449876,3.467586e+06,...,6.068511e+07,20008.857194,1.305230e+07,9.208770e+07,5.198711e+06,3.939701e+06,3.635226e+07,7.786658e+06,3.196435e+07,7.001720e+07
8,8,1.681650e+06,5.468728e+07,5.796416e+06,1.023213e+07,90817.096041,3.951481e+06,4.834578e+07,901717.083675,3.443791e+06,...,6.026868e+07,19871.555738,1.296273e+07,9.145579e+07,5.163037e+06,3.912667e+06,3.610281e+07,7.733226e+06,3.174501e+07,6.953674e+07
9,9,1.670111e+06,5.431203e+07,5.756642e+06,1.016192e+07,90193.934733,3.924367e+06,4.801404e+07,895529.755279,3.420161e+06,...,5.985514e+07,19735.202725,1.287379e+07,9.082825e+07,5.127610e+06,3.885819e+06,3.585508e+07,7.680163e+06,3.152718e+07,6.905959e+07


In [ ]:
svp.plot_emissions_stack(
    d
)